In [2]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, Tool
from duckduckgo_search import DDGS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
import re
import os
from dotenv import load_dotenv

In [3]:
load_dotenv("secrets.env")
my_key = os.getenv("GEMINI_KEY")

os.environ["GOOGLE_API_KEY"] = my_key
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7)

In [4]:
TRUSTED_SITES = [
    "cdc.gov", 
    "nih.gov",
    "drugs.com",
    "medlineplus.gov",
    "hopkinsmedicine.org",
    "who.int",
    "fda.gov",
    "health.harvard.edu"
]

In [5]:
def medical_search(query: str, max_results: int = 5) -> str:
    filter_domains = " OR ".join([f"site:{domain}" for domain in TRUSTED_SITES])
    full_query = f"{query} {filter_domains}"

    results_text = []
    with DDGS() as ddgs:
        for r in ddgs.text(full_query, max_results=max_results):
            title = r.get("title", "")
            snippet = r.get("body", "")
            url = r.get("href", "")
            results_text.append(f"Title: {title}\nSnippet: {snippet}\nSource: {url}")

    return "\n\n".join(results_text) if results_text else "No trusted medical sources found."

In [6]:
tools = [
    Tool(
        name="TrustedMedicalSearch",
        func=medical_search,
        description="Searches medical information from trusted sources like CDC, NIH, Mayo Clinic, and more."
    )
]

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [("system", (
        "You are a medical assistant. You have access to a specialized web search tool "
        "that ONLY returns information from a predefined list of trustworthy sources, "
        "including reputable general, academic, and medical websites. "
        "Use this tool when you need to find current or specific information. "
        "When you provide an answer based on information from the search tool, "
        "If the tool returns 'No information found from trusted sources', state that clearly and do not invent information."
        )),
        ("user", "{input}"),
        AIMessage(content="Okay, I will use the trusted search tool if necessary to find the answer from reliable sources."),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [8]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [9]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, return_intermediate_steps=True)

In [12]:
user_input = "What is Ozempic?"

In [13]:
response = agent_executor.invoke({"input": user_input})
print(f"{response['output']}") 



> Entering new AgentExecutor chain...

Invoking: `TrustedMedicalSearch` with `Ozempic`


Title: Ozempic: Uses, Dosage, Side Effects, Warnings - Drugs.com
Snippet: Ozempic works (mechanism of action) by binding and activating GLP-1 receptors, which increases insulin release, slows stomach emptying, and reduces sugar production by the liver. This results in improved blood sugar levels (HbA1c levels), prolonged fullness, and controlled appetite, which may result in weight loss. ...
Source: https://www.drugs.com/ozempic.html

Title:  Ozempic PI - National Institutes of Health
Snippet: OZEMPIC® causes thyroid C-cell tumors, including medullary thyroid carcinoma (MTC), in humans as the human relevance of semaglutide-induced rodent thyroid C-cell tumors has not been determined (5.1, 13.1). • ®OZEMPIC is contraindicated in patients with a personal or family history of
Source: https://rsc.niaid.nih.gov/sites/default/files/1.14.2.2-package-insert-ozempic.pdf

Title: FDA warns consumers not to 

In [33]:
if 'intermediate_steps' in response and response['intermediate_steps']:
    print("\n--- Agent's Intermediate Steps (Context & Raw Tool Output) ---")
    for step_number, (action, observation) in enumerate(response['intermediate_steps']):
        print(f"Step {step_number + 1}:")
        print(f"  Action Tool: {action.tool}")
        print(f"  Action Input: {action.tool_input}")
        print(f"  Observation (Raw Tool Output):\n{observation}\n---")


--- Agent's Intermediate Steps (Context & Raw Tool Output) ---
Step 1:
  Action Tool: TrustedMedicalSearch
  Action Input: Ozempic side effects
  Observation (Raw Tool Output):
Title: Ozempic Side Effects: Common, Severe, Long Term - Drugs.com
Snippet: Learn about the side effects of Ozempic (semaglutide), from common to rare, for consumers and healthcare professionals.
Source: https://www.drugs.com/sfx/ozempic-side-effects.html

Title: DailyMed - OZEMPIC- semaglutide injection, solution
Snippet: OZEMPIC (semaglutide) injection, for subcutaneous use, contains semaglutide, a human GLP-1 receptor agonist (or GLP-1 analog). The peptide backbone is produced by yeast fermentation.
Source: https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=adec4fd2-6858-4c99-91d4-531f5f2a2d79

Title: PDF
Snippet: The developmental and health benefits of breastfeeding should be considered along with the mother's clinical need for OZEMPIC® and any potential adverse effects on the breastfed infant from O